In [ ]:
pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import keras
from tqdm import tqdm
import pickle
from keras.models import Model
import keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
import itertools
from keras.models import load_model
from sklearn.utils import shuffle
from transformers import BertTokenizer, TFBertModel, BertConfig

In [ ]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset.csv', encoding='latin-1')

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
print('File has {} rows and {} columns'.format(data.shape[0],data.shape[1]))

File has 50000 rows and 2 columns


In [ ]:
import transformers  
from transformers import BertTokenizer, TFBertModel, BertConfig
from transformers import BertTokenizer, TFBertForSequenceClassification

In [ ]:
num_classes=len(data.sentiment.unique())
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=num_classes)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sent= 'how to train the model, lets look at how a trained model calculates its prediction.'

In [ ]:
#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
tokenized_sequence= bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =30,pad_to_max_length = True,return_attention_mask = True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
tokenized_sequence

{'input_ids': [101, 2129, 2000, 3345, 1996, 2944, 1010, 11082, 2298, 2012, 2129, 1037, 4738, 2944, 18422, 2015, 2049, 17547, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
bert_tokenizer.decode(tokenized_sequence['input_ids'])

'[CLS] how to train the model, lets look at how a trained model calculates its prediction. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
data['label'] = data['sentiment'].map({'positive':0,'negative':1})
data.head()

,review,sentiment,label
0,One of the other reviewers has mentioned that ...,positive,0
1,A wonderful little production. <br /><br />The...,positive,0
2,I thought this was a wonderful way to spend ti...,positive,0
3,Basically there's a family where a little boy ...,negative,1
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,0


In [ ]:
sentences=data['review']
labels=data['label']
len(sentences),len(labels)

(50000, 50000)

In [ ]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =256,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

In [ ]:
len(input_ids),len(attention_masks),len(labels)

(50000, 50000, 50000)

In [ ]:
print('Preparing the pickle file.....')

pickle_inp_path='/content/drive/MyDrive/data.pkl'
pickle_mask_path='/content/drive/MyDrive/data.pkl'
pickle_label_path='/content/drive/MyDrive/data.pkl'

Preparing the pickle file.....


In [ ]:
pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((labels),open(pickle_label_path,'wb'))


print('Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_label_path)

Pickle files saved as  /content/drive/MyDrive/data.pkl /content/drive/MyDrive/data.pkl /content/drive/MyDrive/data.pkl


In [ ]:
print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))

Loading the saved pickle files..
Input shape (50000,) Attention mask shape (50000,) Input label shape (50000,)


In [ ]:
from sklearn.model_selection import train_test_split
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))

Train inp shape (40000,) Val input shape (10000,)
Train label shape (40000,) Val label shape (10000,)
Train attention mask shape (40000,) Val attention mask shape (10000,)


In [ ]:
model_save_path='/content/drive/MyDrive/bert_model.h5'

In [ ]:
log_dir='/content/drive/MyDrive'
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)
bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metric])
bert_model.summary()

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_151 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=bert_model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=4,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/4
1250/1250 [==============================] - 148s 109ms/step - loss: 0.0583 - accuracy: 0.9632 - val_loss: 7.6469e-05 - val_accuracy: 1.0000
Epoch 2/4
1250/1250 [==============================] - 133s 106ms/step - loss: 9.0556e-05 - accuracy: 1.0000 - val_loss: 2.3007e-05 - val_accuracy: 1.0000
Epoch 3/4
1250/1250 [==============================] - 133s 106ms/step - loss: 3.2441e-05 - accuracy: 1.0000 - val_loss: 1.0312e-05 - val_accuracy: 1.0000
Epoch 4/4
1250/1250 [==============================] - 133s 107ms/step - loss: 1.5034e-05 - accuracy: 1.0000 - val_loss: 5.2452e-06 - val_accuracy: 1.0000


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
import numpy as np

In [ ]:
#Ignore warnings
import warnings
warnings.filterwarnings('ignore')
trained_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights('/content/drive/MyDrive/bert_model.h5')
preds = trained_model.predict([val_inp,val_mask],batch_size=32)
pred_labels = np.argmax(preds.logits, axis=1)
f1 = f1_score(val_label,pred_labels)
print('F1 score',f1)
print('Classification Report')
print(classification_report(val_label,pred_labels))

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


F1 score 1.0
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4996
           1       1.00      1.00      1.00      5004

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(sentences, labels, test_size=.2)

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_209']
You should probably TRAIN this model on a down-stream task to be able to use 

In [ ]:
model_save_path='/content/drive/MyDrive/distilbert_model.h5'

In [ ]:
log_dir='/content/drive/MyDrive'
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)
model.compile(loss=loss,optimizer=optimizer,metrics=[metric])
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  1538      
_________________________________________________________________
dropout_209 (Dropout)        multiple                  0         
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2=model.fit([train_inp,train_mask],train_label,batch_size=32,epochs=2,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/2
1250/1250 [==============================] - 82s 66ms/step - loss: 1.0784e-06 - accuracy: 1.0000 - val_loss: 1.7896e-07 - val_accuracy: 1.0000
Epoch 2/2
1250/1250 [==============================] - 82s 65ms/step - loss: 4.5955e-07 - accuracy: 1.0000 - val_loss: 5.9652e-08 - val_accuracy: 1.0000


In [ ]:
from transformers import TFDistilBertForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

In [ ]:
#Ignore warnings
import warnings
warnings.filterwarnings('ignore')
trained_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights('/content/drive/MyDrive/distilbert_model.h5')
preds = trained_model.predict([val_inp,val_mask],batch_size=32)
pred_labels = np.argmax(preds.logits, axis=1)
f1 = f1_score(val_label,pred_labels)
print('F1 score',f1)
print('Classification Report')
print(classification_report(val_label,pred_labels))

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_229']
You should probably TRAIN this model on a down-stream task to be able to use 

F1 score 1.0
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4996
           1       1.00      1.00      1.00      5004

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



In [ ]:
from transformers import RobertaConfig, RobertaModel

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [ ]:
from transformers import logging

logging.set_verbosity_warning()

In [ ]:
from transformers import RobertaTokenizer, TFRobertaModel
tokenizer = TFAutoModelForSequenceClassification.from_pretrained('roberta-base')
model3 = TFAutoModelForSequenceClassification.from_pretrained('roberta-base')

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_save_path='/content/drive/MyDrive/robertabert_model.h5'

In [ ]:
log_dir='/content/drive/MyDrive'
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

In [ ]:
from transformers import RobertaForSequenceClassification
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)
model3.compile(loss=loss,optimizer=optimizer,metrics=[metric])
model3.summary()

Model: "tf_roberta_for_sequence_classification_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124055040 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  592130    
Total params: 124,647,170
Trainable params: 124,647,170
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3=model3.fit([train_inp,train_mask],train_label,batch_size=32,epochs=2,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/2
1250/1250 [==============================] - 163s 122ms/step - loss: 0.0089 - accuracy: 0.9978 - val_loss: 2.1280e-05 - val_accuracy: 1.0000
Epoch 2/2
1250/1250 [==============================] - 140s 112ms/step - loss: 3.2272e-05 - accuracy: 1.0000 - val_loss: 6.4971e-06 - val_accuracy: 1.0000


In [ ]:
#Ignore warnings
import warnings
warnings.filterwarnings('ignore')
trained_model = TFAutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels=2)
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights('/content/drive/MyDrive/robertabert_model.h5')
preds = trained_model.predict([val_inp,val_mask],batch_size=32)
pred_labels = np.argmax(preds.logits, axis=1)
f1 = f1_score(val_label,pred_labels)
print('F1 score',f1)
print('Classification Report')
print(classification_report(val_label,pred_labels))

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


F1 score 1.0
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4996
           1       1.00      1.00      1.00      5004

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



In [ ]:
!pip install -U transformers
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [ ]:
from transformers import TFAutoModelForSequenceClassification
tokenizer = TFAutoModelForSequenceClassification.from_pretrained('xlnet-base-cased')

In [ ]:
from transformers import TFAutoModelForSequenceClassification, XLNetModel
PRE_TRAINED_MODEL_NAME = 'xlnet-base-cased'
tokenizer = TFAutoModelForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME)

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj', 'sequence_summary']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
xlnet = TFAutoModelForSequenceClassification.from_pretrained('xlnet-base-cased')

Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj', 'sequence_summary']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_save_path='/content/drive/MyDrive/XLnet_model.h5'

In [ ]:
log_dir='/content/drive/MyDrive'
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)
xlnet.compile(loss=loss,optimizer=optimizer,metrics=[metric])
xlnet.summary()

Model: "tfxl_net_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
transformer (TFXLNetMainLaye multiple                  116718336 
_________________________________________________________________
sequence_summary (TFSequence multiple                  590592    
_________________________________________________________________
logits_proj (Dense)          multiple                  1538      
Total params: 117,310,466
Trainable params: 117,310,466
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model4=xlnet.fit([train_inp,train_mask],train_label,batch_size=32,epochs=2,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/2
1250/1250 [==============================] - 152s 112ms/step - loss: 0.0051 - accuracy: 0.9988 - val_loss: 4.0531e-06 - val_accuracy: 1.0000
Epoch 2/2
1250/1250 [==============================] - 146s 117ms/step - loss: 1.6552e-05 - accuracy: 1.0000 - val_loss: 1.6094e-06 - val_accuracy: 1.0000
